In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
pd.options.display.max_colwidth = 100

In [3]:
df = pd.read_csv('compounds.csv',header = None)

FileNotFoundError: File b'compounds.csv' does not exist

In [ ]:
view_stats = df.iloc[0,:]

In [ ]:
print("so we have " + str(view_stats[0]) +" view counts, "+str(view_stats[1])+" likes, "+str(view_stats[2])+" dislikes. ")

We can also try to see how it looks like by creating a pie-chart

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

labels = ['Likecout','Dislikecount']
values = [view_stats[1],view_stats[2]]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

So the cahrt above can present **visualisation** of how likeable our video is.

In [ ]:
df = df.drop(index = 0)


In [ ]:
df = df.rename(columns = {0:'text',1:'score',2:'likecount'})

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
def Clean_Text(text):
    """
    text: input for the text
    Disclaimer:
    This is a ongoing cleaning function
    """

    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"he's","he is ",text)
    text = re.sub(r"she's","she is ",text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'[^\w\s]','',text)
    text = text.strip(' ')
    return text


In [ ]:
df['clean_text'] = df['text'].map(lambda com:Clean_Text(com))

In [ ]:
df.head(10)


So now since we have `cleaned text`, we are going to try use NLTK again to see the new score.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sid
sid = sid()
df['clean_text_score'] = df['clean_text'].map(lambda com:sid.polarity_scores(com)['compound'])

In [ ]:
df.head(3)

What comments is the most likeable one ? And what's the score it get from the `polarity_scores` function ? 

In [ ]:
df_like_ranking = df.sort_values('likecount',ascending = False)

In [ ]:
df_like_ranking.head(10)

If you wanna know **all** the likecount in a decending order ,just remove the `#` in the next chuck and it will show you the result.

In [ ]:
#df_like_ranking

Of course the `polaity_scores` function itself can have some issues on giving score to the online comments ,some of them contains
**sarcasm and slang** , those together all cause the fucntion to pick up and give scores. So for this, it's really important to see the **intersection of the most likeable and the highest score**.

In [ ]:
df.info()

In [ ]:
def pick_up_comments(data,score,threshold):
    """
    This function take three inputs:
    - data :dataframe
    - score : score you wannt to examine,input requires to be in [-1,1]. For negative numbers it will run scores no greater than the input,for positve
        numbers it will find scores no smaller than the input.
    - likecount : postive interger input required.
    
    """
    
    if score > 0:
        df =  data.loc[(df_like_ranking["clean_text_score"] >= score) & (df_like_ranking["likecount"] >= threshold)].sort_values(['likecount', 'score'],ascending=[False, False])
        return df
    elif score <= 0:
        df =  data.loc[(df_like_ranking["clean_text_score"] <= score) & (df_like_ranking["likecount"] >= threshold)].sort_values(['likecount', 'score'],ascending=[False, True])
        return df
              

----

Here is just a part we are going to use the function to see how the `clean_text_score` interact with likecount.
You can change the number for your own analysis.

In [4]:
pick_up_comments(df_like_ranking,0.8,50)

NameError: name 'pick_up_comments' is not defined

In [5]:
pick_up_comments(df_like_ranking,-0.5,50)

NameError: name 'pick_up_comments' is not defined

In [6]:
pick_up_comments(df_like_ranking,-0.4,50)

NameError: name 'pick_up_comments' is not defined

In [7]:
pick_up_comments(df_like_ranking,-0.6,20)

NameError: name 'pick_up_comments' is not defined

We can play with the function if we want to see how the comments distribute. 

For this special case, specific analysis and detailed explanation would be useful for this single one video, so I will leave here .

-----

One thing I am interested in is the distribution of likecount of certain videos , let's check .

# Histogram of the score

In [8]:
from scipy.stats import kurtosis
doanes = lambda data: int(1 + np.log(len(data)) + np.log(1 +  kurtosis(data) * (len(data)/6.)** 0.5))

In [9]:
df_like_ranking.clean_text_score.hist()

NameError: name 'df_like_ranking' is not defined

# Histogram of the score

In [10]:
from scipy.stats import kurtosis
doanes = lambda data: int(1 + np.log(len(data)) + np.log(1 +  kurtosis(data) * (len(data)/6.)** 0.5))

In [11]:
df_like_ranking.likecount.hist(bins = doanes(df_like_ranking["likecount"]))

NameError: name 'df_like_ranking' is not defined

maybe try to plot scatter points betweeen `likecounts` and `clean_text_score`.

In [12]:
sns.jointplot(x="clean_text_score", y="likecount", data=df_like_ranking)

NameError: name 'df_like_ranking' is not defined